In [ ]:
from flask import Flask, jsonify, request, render_template_string
from pyngrok import ngrok
from flask_cors import CORS
import requests

ngrok.set_auth_token('2lzFTKGguCaQnvqkBnScBWcsxAP_4srWnM1hngHbUPyDLQcw')

app = Flask(__name__)
CORS(app)

recognized_text = ""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Recognized Text Viewer</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f9;
            color: #333;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .container {
            width: 90%;
            max-width: 600px;
            background: #ffffff;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            padding: 20px 30px;
            text-align: center;
        }
        h1 {
            font-size: 1.8rem;
            margin-bottom: 20px;
            color: #4a90e2;
        }
        #recognizedText {
            white-space: pre-wrap;
            border: 1px solid #e0e0e0;
            border-radius: 5px;
            padding: 15px;
            background-color: #f9f9f9;
            color: #444;
            font-size: 1.1rem;
            text-align: left;
            min-height: 100px;
            overflow-y: auto;
        }
        .footer {
            margin-top: 20px;
            font-size: 0.9rem;
            color: #888;
        }
    </style>
    <script>
        async function fetchRecognizedText() {
            const response = await fetch('/get-recognized-text');
            const data = await response.json();
            document.getElementById('recognizedText').innerText = data.recognized_text || "No text received yet.";
        }
        setInterval(fetchRecognizedText, 1000);
    </script>
</head>
<body>
    <div class="container">
        <h1>Recognized Text Viewer</h1>
        <div id="recognizedText">Loading...</div>
        <div class="footer">
            <p>Updated dynamically every 2 seconds</p>
        </div>
    </div>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/get-recognized-text', methods=['GET'])
def get_recognized_text():
    global recognized_text
    return jsonify({"recognized_text": recognized_text}), 200

@app.route('/data', methods=['POST'])
def receive_data():
    global recognized_text
    data = request.get_json()
    recognized_text = data.get('recognizedText', '')
    print(f"Received recognized text: {recognized_text}")
    return jsonify({"message": "Recognized text received successfully!", "received_text": recognized_text}), 200

@app.route('/send-to-arduino', methods=['POST'])
def send_to_arduino():
    global recognized_text
    arduino_ip = "ARDUINO's IP"  
    arduino_port = "# PORT"            
    url = f"http://{arduino_ip}:{arduino_port}/receive"
    try:
        response = requests.post(url, json={"data": recognized_text})
        return jsonify({"message": "Data sent to Arduino successfully!", "response": response.text}), response.status_code
    except requests.exceptions.RequestException as e:
        return jsonify({"message": "Failed to send data to Arduino", "error": str(e)}), 500

public_url = ngrok.connect(5000)
print(f"Flask app is publicly accessible at: {public_url}")

app.run(port=5000)
